In [1]:
import pandas as pd
import quantstats as qs

dados_empresas = pd.read_csv("dados_empresas.csv")
ibov = pd.read_csv('ibov.csv')
dados_empresas['retorno'] = dados_empresas.groupby('ticker')['preco_fechamento_ajustado'].pct_change()
dados_empresas = dados_empresas[dados_empresas['volume_negociado'] > 100000]
dados_empresas["ranking_ebit_ev"] = dados_empresas.groupby('data')['ebit_ev'].rank(ascending=False)
dados_empresas["ranking_roic"] = dados_empresas.groupby('data')['roic'].rank(ascending=False)
dados_empresas['ranking_final'] = dados_empresas['ranking_ebit_ev'] + dados_empresas['ranking_roic']
dados_empresas['ranking_final'] = dados_empresas.groupby('data')["ranking_final"].rank(ascending = True)


dados_empresas = dados_empresas[dados_empresas['ranking_final'] <= 10]
rentabilidade_por_carteiras =  dados_empresas.groupby('data')['retorno'].mean()
rentabilidade_por_carteiras = rentabilidade_por_carteiras.to_frame() 
rentabilidade_por_carteiras['Magic Formula'] = (rentabilidade_por_carteiras['retorno'] + 1).cumprod() - 1
rentabilidade_por_carteiras = rentabilidade_por_carteiras.shift(1)
rentabilidade_por_carteiras = rentabilidade_por_carteiras.dropna()

retornos_ibov = ibov['fechamento'].pct_change().dropna()
retornos_acum_ibov = (1 + retornos_ibov).cumprod() - 1
# Alinha os índices de datas para evitar erro de shape
rentabilidade_por_carteiras['Ibovespa'] = retornos_acum_ibov.loc[rentabilidade_por_carteiras.index]
rentabilidade_por_carteiras = rentabilidade_por_carteiras.drop('retorno', axis = 1)

qs.extend_pandas()
rentabilidade_por_carteiras.index = pd.to_datetime(rentabilidade_por_carteiras.index)
rentabilidade_por_carteiras['Magic Formula'].plot_monthly_heatmap()
rentabilidade_por_carteiras['Ibovespa'].plot_monthly_heatmap()


KeyError: "None of [Index(['2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30',\n       '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30',\n       ...\n       '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31', '2025-01-31',\n       '2025-02-28', '2025-03-31', '2025-04-30', '2025-05-31', '2025-06-30'],\n      dtype='object', name='data', length=113)] are in the [index]"